# Pyspark End-To-End

`Course - https://www.udemy.com/course/pyspark-end-to-end-developer-course-spark-with-python/learn/lecture/37180040#overview`

## Topics -
    - How to read flat file and tables using jbdc connection
    - DataFrame API Selection
    - DataFrame Where & Filter
    - DataFrame API Sorting
    - DataFrame API Set
    - DataFrame API Join

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317146 sha256=7a84452b4f95251328d8d8b87a35ff2bd11d3a1ee5dd88f671cf9557672c6125
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
import pyspark
from pyspark.sql import SparkSession,functions
from pyspark.sql.functions import *
from pyspark.sql.types import StructField,StructType,IntegerType,StringType
spark = SparkSession.builder.appName("Pyspark-Tutorial").getOrCreate()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/16 11:08:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/kaggle/input/retail-db-tutorial/RetailDB SalesData/Customers/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Orders/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Products/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Order_items/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Categories/part-00000
/kaggle/input/retail-db-tutorial/RetailDB SalesData/Departments/part-00000


## How to read file using various techniques

In [3]:
ord_file_path = "/kaggle/input/retail-db-tutorial/RetailDB SalesData/Orders"
ord = spark.read.load(ord_file_path,sep=",",format='csv',schema=('ordid int,ord_date timestamp,ord_cust_id int, ord_status string'))
ord.show(1)

ord = spark.read.csv(ord_file_path,sep=",",schema=('ordid int,ord_date timestamp,ord_cust_id int, ord_status string'))
ord.show(1)

ord = spark.read.format('csv').load(ord_file_path,sep=',',schema='ordid int,ord_date timestamp,ord_cust_id int, ord_status string')
ord.show(1)

+-----+-------------------+-----------+----------+
|ordid|           ord_date|ord_cust_id|ord_status|
+-----+-------------------+-----------+----------+
|    1|2013-07-25 00:00:00|      11599|    CLOSED|
+-----+-------------------+-----------+----------+
only showing top 1 row

+-----+-------------------+-----------+----------+
|ordid|           ord_date|ord_cust_id|ord_status|
+-----+-------------------+-----------+----------+
|    1|2013-07-25 00:00:00|      11599|    CLOSED|
+-----+-------------------+-----------+----------+
only showing top 1 row

+-----+-------------------+-----------+----------+
|ordid|           ord_date|ord_cust_id|ord_status|
+-----+-------------------+-----------+----------+
|    1|2013-07-25 00:00:00|      11599|    CLOSED|
+-----+-------------------+-----------+----------+
only showing top 1 row



## How to read table and jdbc connection

spark.read.jdbc("connection string / jdbc:mysql://localhost:3306/emp","table",
                properties={"user":"root","password":"root","driver":"com.mysql.cj.jdbc.Driver"})

spark.read.format("jdbc") \
.option("url","connection string") \
.option("driver","com.mysql.cj.jdbc.Driver") \
.option("user","root")
.option("password":"root")
.option("dbtable","employee")
# .option("dbtable","select empid, salary from employee where gender = 'M' ")
# .option("query"," select empid, salary from employee where gender = 'M' ")
.load()

# DataFrame API Selection

### Select
Select one or more columns
Applying necessary functions on select

### SelectExpr
This is a variant of select that accepts SQL expressions.
If we want to use any functions available in SQL but not in Spark Built-in functions, then we can use selectExpr
            -- explode

### withColumn(colName, col)
Applied transformation to only selected columns.
The first argument is an alias name. If we give an alias name same as a column name, the transformations will apply
on the same column.
Otherwise a new column will be formed. Avoid giving alias name same as column name.
ord.withColumn('order_month',substring(orderDF.order_date,1,10)).show()


### withColumnRenamed(existingCol, newCol)
Rename Existing Column.
ord.withColumnRenamed('order_id','order_id1').show()

### drop(*cols)
Drop a column.
order = ord.drop('order_id','order_date')

### dropDuplicates(subset=None)
Drop duplicate rows.
Optionally can consider only subset of columns.
emp.dropDuplicates().show()
emp.dropDuplicates(("name","score1","score2")).show()

In [4]:
ord
data = [('Robert',35,40,40),('Robert',35,40,40),('Ram',31,33,29),('Ram',31,33,91)]
emp = spark.createDataFrame(data=data,schema=['name','score1','score2','score3'])

In [5]:
ord.select("ordid",'ordid',ord.ordid).show(5)

ord.select(lower("ord_status").alias("lower_sts"),"ord_status").show(5)

+-----+-----+-----+
|ordid|ordid|ordid|
+-----+-----+-----+
|    1|    1|    1|
|    2|    2|    2|
|    3|    3|    3|
|    4|    4|    4|
|    5|    5|    5|
+-----+-----+-----+
only showing top 5 rows

+---------------+---------------+
|      lower_sts|     ord_status|
+---------------+---------------+
|         closed|         CLOSED|
|pending_payment|PENDING_PAYMENT|
|       complete|       COMPLETE|
|         closed|         CLOSED|
|       complete|       COMPLETE|
+---------------+---------------+
only showing top 5 rows



In [6]:
# ord.show(5)
ord.selectExpr("substring(ord_date,1,4) as ord_year","ord_date") \
.selectExpr("ord_year","substring(ord_date,6,2) as ord_month","ord_date") \
.selectExpr("ord_date","ord_month","ord_year","substring(ord_date,9,2) as ord_day") \
.select("ord_year","ord_month","ord_date") \
.show(5)

+--------+---------+-------------------+
|ord_year|ord_month|           ord_date|
+--------+---------+-------------------+
|    2013|       07|2013-07-25 00:00:00|
|    2013|       07|2013-07-25 00:00:00|
|    2013|       07|2013-07-25 00:00:00|
|    2013|       07|2013-07-25 00:00:00|
|    2013|       07|2013-07-25 00:00:00|
+--------+---------+-------------------+
only showing top 5 rows



In [7]:
ord.withColumn("ord_year",substring("ord_date",1,4)).show(5)

ord.withColumn("ord_date",substring("ord_date",1,4)).show(5)

+-----+-------------------+-----------+---------------+--------+
|ordid|           ord_date|ord_cust_id|     ord_status|ord_year|
+-----+-------------------+-----------+---------------+--------+
|    1|2013-07-25 00:00:00|      11599|         CLOSED|    2013|
|    2|2013-07-25 00:00:00|        256|PENDING_PAYMENT|    2013|
|    3|2013-07-25 00:00:00|      12111|       COMPLETE|    2013|
|    4|2013-07-25 00:00:00|       8827|         CLOSED|    2013|
|    5|2013-07-25 00:00:00|      11318|       COMPLETE|    2013|
+-----+-------------------+-----------+---------------+--------+
only showing top 5 rows

+-----+--------+-----------+---------------+
|ordid|ord_date|ord_cust_id|     ord_status|
+-----+--------+-----------+---------------+
|    1|    2013|      11599|         CLOSED|
|    2|    2013|        256|PENDING_PAYMENT|
|    3|    2013|      12111|       COMPLETE|
|    4|    2013|       8827|         CLOSED|
|    5|    2013|      11318|       COMPLETE|
+-----+--------+-----------+--

In [8]:
ord.withColumnRenamed("ord_status","ord_sts").show(5)

+-----+-------------------+-----------+---------------+
|ordid|           ord_date|ord_cust_id|        ord_sts|
+-----+-------------------+-----------+---------------+
|    1|2013-07-25 00:00:00|      11599|         CLOSED|
|    2|2013-07-25 00:00:00|        256|PENDING_PAYMENT|
|    3|2013-07-25 00:00:00|      12111|       COMPLETE|
|    4|2013-07-25 00:00:00|       8827|         CLOSED|
|    5|2013-07-25 00:00:00|      11318|       COMPLETE|
+-----+-------------------+-----------+---------------+
only showing top 5 rows



In [9]:
ord.drop('ord_status','ord_date').show(2)
ord.drop('ord_status').show(2)

+-----+-----------+
|ordid|ord_cust_id|
+-----+-----------+
|    1|      11599|
|    2|        256|
+-----+-----------+
only showing top 2 rows

+-----+-------------------+-----------+
|ordid|           ord_date|ord_cust_id|
+-----+-------------------+-----------+
|    1|2013-07-25 00:00:00|      11599|
|    2|2013-07-25 00:00:00|        256|
+-----+-------------------+-----------+
only showing top 2 rows



In [10]:
emp.show()

emp.dropDuplicates().show() ## only 1 record filtered

emp.dropDuplicates(["name"]).show()

emp.dropDuplicates(["name","score2"]).show()

+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
|   Ram|    31|    33|    91|
+------+------+------+------+

+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
|   Ram|    31|    33|    91|
+------+------+------+------+

+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
+------+------+------+------+

+------+------+------+------+
|  name|score1|score2|score3|
+------+------+------+------+
|Robert|    35|    40|    40|
|   Ram|    31|    33|    29|
+------+------+------+------+



# DataFrame API Filter

### filter(condition): (Its alias ‘where’)
Filter rows using a given condition

use '&' for 'and‘. '|' for 'or‘. (boolean expressions)

Use column function isin() for multiple search Or use IN Operator for SQL Style syntax.

In [11]:
# &
ord.where((ord.ordid > 10) & (ord.ordid < 20)).show(5)

ord.filter((ord.ordid > 10) & (ord.ordid < 20)).show(5)

# |
ord.where((ord.ordid == 10) | (ord.ordid == 20)).show(5)

ord.where((ord.ordid == 10) | (ord.ordid == 20)).show(5)

# isin
ord.where(ord.ord_status.isin("CLOSED","PENDING")).show(5)
ord.filter(ord.ord_status.isin("CLOSED","PENDING")).show(5)

# sql like where optional
ord.where("ord_status in ('CLOSED','NOT-PENDING') ").show(5)

+-----+-------------------+-----------+---------------+
|ordid|           ord_date|ord_cust_id|     ord_status|
+-----+-------------------+-----------+---------------+
|   11|2013-07-25 00:00:00|        918| PAYMENT_REVIEW|
|   12|2013-07-25 00:00:00|       1837|         CLOSED|
|   13|2013-07-25 00:00:00|       9149|PENDING_PAYMENT|
|   14|2013-07-25 00:00:00|       9842|     PROCESSING|
|   15|2013-07-25 00:00:00|       2568|       COMPLETE|
+-----+-------------------+-----------+---------------+
only showing top 5 rows

+-----+-------------------+-----------+---------------+
|ordid|           ord_date|ord_cust_id|     ord_status|
+-----+-------------------+-----------+---------------+
|   11|2013-07-25 00:00:00|        918| PAYMENT_REVIEW|
|   12|2013-07-25 00:00:00|       1837|         CLOSED|
|   13|2013-07-25 00:00:00|       9149|PENDING_PAYMENT|
|   14|2013-07-25 00:00:00|       9842|     PROCESSING|
|   15|2013-07-25 00:00:00|       2568|       COMPLETE|
+-----+----------------

# DataFrame APIs Sort


### sort() or orderBy()
orderBy() is an alias for .sort()
Sort specific column(s).

### sortWithinPartitions
At time, we may not want sort globally, but with in a group. In that case we can use sortWithinPartitions.

In [12]:
ord.sort(ord.ord_status.desc()).show(2)
ord.sort(col("ord_status").desc()).show(2)
ord.sort("ord_status","ord_cust_id").show(2)
ord.sort("ord_status","ord_cust_id",ascending=[0,1]).show(2)

+-----+-------------------+-----------+---------------+
|ordid|           ord_date|ord_cust_id|     ord_status|
+-----+-------------------+-----------+---------------+
|   69|2013-07-25 00:00:00|       2821|SUSPECTED_FRAUD|
|  246|2013-07-26 00:00:00|       9616|SUSPECTED_FRAUD|
+-----+-------------------+-----------+---------------+
only showing top 2 rows

+-----+-------------------+-----------+---------------+
|ordid|           ord_date|ord_cust_id|     ord_status|
+-----+-------------------+-----------+---------------+
|   69|2013-07-25 00:00:00|       2821|SUSPECTED_FRAUD|
|  246|2013-07-26 00:00:00|       9616|SUSPECTED_FRAUD|
+-----+-------------------+-----------+---------------+
only showing top 2 rows

+-----+-------------------+-----------+----------+
|ordid|           ord_date|ord_cust_id|ord_status|
+-----+-------------------+-----------+----------+
|19709|2013-11-24 00:00:00|         18|  CANCELED|
|11256|2013-10-02 00:00:00|         35|  CANCELED|
+-----+----------------

In [13]:
data = [("a",1),("b",2),("c",3),("d",4),("e",5)]
df = spark.createDataFrame(data,schema='col1 string,col2 int')
df = df.repartition(2)
df = df.withColumn("partid",spark_partition_id())

df.sort(df.col1).show()
df.sortWithinPartitions(df.col1).show()

+----+----+------+
|col1|col2|partid|
+----+----+------+
|   a|   1|     1|
|   b|   2|     1|
|   c|   3|     0|
|   d|   4|     0|
|   e|   5|     1|
+----+----+------+

+----+----+------+
|col1|col2|partid|
+----+----+------+
|   c|   3|     0|
|   d|   4|     0|
|   a|   1|     1|
|   b|   2|     1|
|   e|   5|     1|
+----+----+------+



## DataFrame APIs : Set Operators

### union() and unionAll():
Same and contains duplicate values
Use distinct after union or unionAll to remove duplicates

### unionByName():
The difference between this function and :func:`union` is that this function
resolves columns by name (not by position)

### intersect(): 
Containing rows in both DataFrames. Removed duplicates

### intersectAll():
Same as intersect. But retains the duplicates

### exceptAll():
Rows present in one DataFrame but not in another

In [14]:
df1 = spark.range(4)
df2 = spark.range(3,5)

# Union and UnionAll performs same way

df1.union(df2).show()
df1.unionAll(df2).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  3|
|  4|
+---+

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  3|
|  4|
+---+



In [15]:
df1.intersect(df2).show()
df1.intersectAll(df2).show()

+---+
| id|
+---+
|  3|
+---+

+---+
| id|
+---+
|  3|
+---+



## DataFrame APIs : Join

Join Type
inner INNER JOIN, JOIN
outer, full, fullouter,
full_outer
FULL OUTER JOIN
left,
left_outer,leftouter
LEFT JOIN
right,right_outer,rig
htouter
RIGHT JOIN
cross CROSS JOIN
left_anti, leftanti
leftsemi,left_semi

In [16]:
df1 = spark.createDataFrame(data=[(1,'Robert'),(2,'Ria'),(3,'James')],schema='empid int, empname string')
df2 = spark.createDataFrame(data=[(2,'USA'),(4,'India')],schema='empid int, country string')

# Inner Join
df1.join(df2,df1.empid == df2.empid).select(df1.empid.alias("employeeid"),df1.empname,df2.country).show()

# left anti join
# cannot get right table attributes and it executes faster as compared to inner as it doesn't check right/second table attributes
df1.join(df2,df1.empid == df2.empid,"leftanti").show()

# Left outer or Left
df1.join(df2,df1.empid == df2.empid,"left")\
.select(df1.empid,df1.empname,df2.country) \
.sort(df2.country.desc()) \
.show()

# full outer
df1.join(df2,df1.empid == df2.empid,"full")\
.select(df1.empid,df1.empname,df2.country) \
.sort(df1.empid.asc()) \
.show()

+----------+-------+-------+
|employeeid|empname|country|
+----------+-------+-------+
|         2|    Ria|    USA|
+----------+-------+-------+

+-----+-------+
|empid|empname|
+-----+-------+
|    1| Robert|
|    3|  James|
+-----+-------+

+-----+-------+-------+
|empid|empname|country|
+-----+-------+-------+
|    2|    Ria|    USA|
|    1| Robert|   null|
|    3|  James|   null|
+-----+-------+-------+

+-----+-------+-------+
|empid|empname|country|
+-----+-------+-------+
| null|   null|  India|
|    1| Robert|   null|
|    2|    Ria|    USA|
|    3|  James|   null|
+-----+-------+-------+



In [17]:
# self

df1 = spark.createDataFrame(data=((1,'Robert',2),(2,'Ria',3),(3,'James',5)),schema='empid int,empname string,managerid int')
df1.show()
df1.alias("emp1").join(df1.alias("emp2"),col("emp1.managerid") == col("emp2.empid"))\
.select(col("emp1.empid").alias("EmployeeID"),col("emp1.empname").alias("EmployeeName"),col("emp2.empid").alias("ManagerID"),col("emp2.empname").alias("ManagerName")) \
.show()

+-----+-------+---------+
|empid|empname|managerid|
+-----+-------+---------+
|    1| Robert|        2|
|    2|    Ria|        3|
|    3|  James|        5|
+-----+-------+---------+

+----------+------------+---------+-----------+
|EmployeeID|EmployeeName|ManagerID|ManagerName|
+----------+------------+---------+-----------+
|         1|      Robert|        2|        Ria|
|         2|         Ria|        3|      James|
+----------+------------+---------+-----------+

